# **Task 3 - Reichardt Detectors**

<div>
    <img src="https://pbs.twimg.com/media/DV9-v1GW4AgTQoT.png" width=190px; height=230px, align='right'/>
</div>
<!-- https://ars.els-cdn.com/content/image/1-s2.0-S0960982217300738-gr1.jpg -->

This task will create an "Hassenstein-Reichardt detector", i.e. an elementary and hypothetical neural circuit inspired by the fruit fly. Such model is used to detect correlations between adjacent points in order to achieve motion perception. It consists of a cascade of spatial filter and temporal filter applied to adjacent locations on the input space. When an input (light, in case of visual systems) is received within a spatial kernel, a signal is sent and then delayed in time ($\tau$). If the moving object in the scene has the correct (preferred) speed and direction, it will activate the adjacent spatial kernel after a time that equals the previous delay $\tau$. The corresponding signal (with no delay) will then be summed to the delayed signal from the former location, eliciting an output in the final neuron. In case the object moves along the null direction (or with the wrong speed), the two signals will not be coincident and the final detector will thus not reach the firing threshold for emitting an output.

In this implementation, we will use event-based data relative to a moving bar recorded from a neuromorphic sensor. We will not use the whole pixel array of the sensor (346x260) but the central squared region with shape 60x60 and only the ON polarity. The bar may either move rightward (R) or leftward (L) with 3 different speeds (v1, v2, v3). Code for the initial data loading and pre-processing is given. The exercise focuses on building the architecture of the network, which will consist of:
- a first input neural population with 60x60 neurons (pixels);
- a convolutional neural layer of 6 LIF neurons applying spatial filtering with a 2D elongated gaussian kernel;
- two output populations (with 1 neuron each) of motion sensitive LIF neurons taking delayed inputs (thus applying temporal filtering) from neurons in the convolutional layer.

**Run the following cell before you start!**

In [ ]:
!pip3 install dv
!pip3 install brian2
from dv import AedatFile
from brian2 import *
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import rc
rc('animation', html='jshtml')

**In order to use the event-based data stored in the drive folder, you will have to give permissions to this notebook to access google drive.** To do so click the link that appears after you run the cell below. It will redirict you to a page where you will have to confirm your permission and copy the authorization code that appears. Paste the code below and press enter.

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
data_dir = "drive/My Drive/Neuromorphic_Exe20-21/data"
!cp "drive/My Drive/Neuromorphic_Exe20-21/utils/DAVIS_numpy_utils.py" .
!cp "drive/My Drive/Neuromorphic_Exe20-21/utils/kernels.py" .
from DAVIS_numpy_utils import *
from kernels import *

Define some parameters of the event-based data you want to load. Particularly, you may choose between 2 directions and 3 speeds of motion.

In [ ]:
# You may change these variables, related to the motion of the input stimulus:
direction = 'R'    # 'R' or 'L'
speed = 'v3'       # 'v1', 'v2' or 'v3'

# Do not change these variables:
duration = 0.3 * 10 ** 6   # duration of the simulation (us)
N_x, N_y = 346, 260        # shape of the whole DAVIS pixel array
input_size = 60            # shape of the central squared region we will consider

## Load & Process Data

Load the event-based data file (with AEDAT4 extension) containing all outputs from the DAVIS sensor. Store DVS events in a numpy array.

In [ ]:
file = data_dir + "/bar_" + direction + '_' + speed + ".aedat4"
with AedatFile(file) as f:
    # Put all events in a structured numpy array
    events_struct = np.hstack([event_packet for event_packet in f['events'].numpy()])
    # Access information of events by type
    ts, x, y, pol = events_struct['timestamp'], events_struct['x'], events_struct['y'], events_struct['polarity']
    # Put info in a 4d-array with N rows (number of events) and 4 columns (ts, x, y, p)
    events = np.stack((ts, x, y, pol)).T

Show the DVS recording and distinguish between ON and OFF events with colors (green and red respectively).

In [ ]:
dvsvid_on, dvsvid_off = dvs_computevideofreq_onoff(dvs_flattenaddress(events, N_x, N_y), N_x, N_y, rate=50)
minval, maxval = min(dvsvid_on.min(), dvsvid_off.min()), max(dvsvid_on.max(), dvsvid_off.max())
frames_dvs = np.zeros((*dvsvid_on.shape, 3))  # RGB video
frames_dvs[:, :, :, 0] = np.uint8(np.interp(dvsvid_off, (minval, maxval), (0, 255)))
frames_dvs[:, :, :, 1] = np.uint8(np.interp(dvsvid_on, (minval, maxval), (0, 255)))

fig = plt.figure(figsize=(5, 4))
plt.title('DVS video: ON=green, OFF=red')
plt.axis('off')
plot_dvs_frames = []
for frame in frames_dvs:
    dvs_image = plt.imshow(frame.astype(np.uint8), vmin=0, vmax=255, animated=True)
    plot_dvs_frames.append([dvs_image])
plt.close()
animation.ArtistAnimation(fig, plot_dvs_frames, interval=30, blit=True, repeat_delay=1000)

Pre-process the event-based data (mainly, cut the pixel array, cut the recording in the period where the stimulus is present and select ON events only). Then store timestamps and pixel addresses of events in different variables. We will use those as input to the network through a "*SpikeGeneratorGroup*".

In [ ]:
dvs = dvs_cut_pixelarray(events, input_size, N_x, N_y)  # cut the pixel array to a squared region
dvs_flat = dvs_flattenaddress(dvs, input_size, input_size)  # from (x,y) coordinate to flatten index
dvs_flat_on = dvs_select_polarity(dvs_flat, polarity='on')  # select only ON events and remove the others
start = dvs_compute_start(dvs_flat_on, input_size, input_size, duration)  # compute the moment when the stimulus starts to move in the central region
dvs_flat_on = dvs_cut_timewindow(dvs_flat_on, start, start + duration)  # remove all events occurring before 'start' and after 'start + duration'
dvs_flat_on = dvs_timereset(dvs_flat_on)  # reset timestamps so that first event has 0 timestamp
ts, id = dvs_return_id_and_ts(dvs_flat_on)  # separate timestamps and flattened-address of events' array
ts, id = dvs_refractory_filter(address=id, timestamp=ts, refractory=100)  # apply refractory filter for a 100 us refractory period

Show the remaining events.

In [ ]:
dvsvid_on = dvs_computevideofreq(dvs_flat_on, input_size, input_size, rate=80)
minval, maxval = dvsvid_on.min(), dvsvid_on.max()
frames_dvs = np.zeros((*dvsvid_on.shape, 3))  # RGB video
frames_dvs[:, :, :, 1] = np.uint8(np.interp(dvsvid_on, (minval, maxval), (0, 255)))

fig = plt.figure(figsize=(5, 4))
plt.title('DVS video: ON=green, OFF=red')
plt.axis('off')
plot_dvs_frames = []
for frame in frames_dvs:
    dvs_image = plt.imshow(frame.astype(np.uint8), vmin=0, vmax=255, animated=True)
    plot_dvs_frames.append([dvs_image])
plt.close()
animation.ArtistAnimation(fig, plot_dvs_frames, interval=80, blit=True, repeat_delay=1000)

## Define Convolution

Given the kernel dimension, convolution information and the shape of the input space, compute the dimension that the convolutional layer (the neural population after spatial filtering) must have.

In [ ]:
kernel_sigma = 4
kernel_elongation = 10
kernel_th = 0.5
stride = 10
kernel_size = compute_dim_gauss2d_elong(input_size, sigma=kernel_sigma, p=kernel_elongation, threshold=kernel_th)
output_size, error_conv = convolution_out_shape(input_size, kernel_size=kernel_size, stride=stride, return_error=True)
print('Information of convolution on input pixel array:\n'
      ' - input shape of pixel array is {}\n'
      ' - kernel dimension is {}\n'
      ' - stride is {}\n'
      ' - output shape of convolutional layer is {}\n'
      ' - error in sliding kernel over input is {}'
      .format(input_size, kernel_size, stride, output_size, round(error_conv, 2)))

Visualise convolution: show the spatial kernel sliding on the input space.

In [ ]:
all_kernels = np.zeros((output_size, input_size ** 2))
y_center = int(round(input_size / 2))
for x in range(output_size):
  x_center = x * stride + int(round((kernel_size - 1) / 2))
  target = np.ravel_multi_index((y_center, x_center), (input_size, input_size))
  # i.e. target =  x_center + y_center * input_size
  all_kernels[x, :] = gauss2d_elong(target, sigma=kernel_sigma, p=kernel_elongation, teta=90, space_dim=input_size, threshold=kernel_th)
all_kernels = all_kernels.reshape(output_size, input_size, input_size)

fig = plt.figure(figsize=(5, 4))
plt.title('Sliding Spatial Kernel')
plt.axis('off')
plot_kernel = []
for kernel in all_kernels:
    kernel_image = plt.imshow(kernel * 255, vmin=0, vmax=255, animated=True)
    plot_kernel.append([kernel_image])
plt.close()
animation.ArtistAnimation(fig, plot_kernel, interval=500, blit=True)

## Build Network Architecture & Run Simulation

Setup the simulation to use 0.1ms timesteps.

In [ ]:
# Write your code here

Create the input neural population (hint: use a "*SpikeGeneratorGroup*"). Neurons *id* must spike at times *ts*.

In [ ]:
# Input layer (events from the sensor)
Layer_input = # ...

Define the LIF neuron model. The resting value is -65mV, the threshold -50mV, the membrane time constant 2ms and the synapse time constant 5ms. Then create convolutional layer with a number of neurons equal to *ouput_size*. The reset potential is equal to the resting value and the refractory period is 150ms. Remember to initialize the membrane potential to the resting value.

In [ ]:
# Define neuron model and parameters
model = # ...

# Convolutional layer
Layer_convolution = # ...

Create the two output population with 1 LIF neuron each. The reset potential is equal to the resting value and the refractory period is 50ms.

In [ ]:
# RIGHTWARD Reichardt detector neuron
Layer_reichardt_R = # ...

# LEFTWARD Reichardt detector neuron
Layer_reichardt_L = # ...

Create the convolutional synapses, which define the spatial filtering between the input population and the convolutional layer. The spatial kernel (shown in the animation above) should shift only along the x dimension of the input space by a number of pixels defined by the variable *stride*. The variables *x_center* and *y_center* define the coordinates of the central pixel of the kernel.

In [ ]:
# Synapses: convolution
Syn_convolution = # ...

y_center = # ...

kernel_weights = np.asarray([])
for x in range(output_size):

    x_center = # ...

    target_pixel = x_center + y_center * input_size
    kernel = gauss2d_elong(target_pixel, sigma=kernel_sigma, p=kernel_elongation, teta=90, space_dim=input_size, threshold=kernel_th)
    indices_kernel = np.where(kernel >= kernel_th)[0]
    Syn_convolution.connect(i=indices_kernel, j=x)
    kernel_weights = np.concatenate((kernel_weights, kernel[indices_kernel]))
Syn_convolution.w = kernel_weights * mV

Create 2 delayed synapses groups (defining the temporal filtering) between the convolution layer and the 2 output (motion sensitive) neurons. Connectivity should be "all-to-all" and all weights should be equal to 10mV. The delay between adjacent spatial filters should be equal to *delay_reichardt*. In case of rightward sensitivity, the delay should decrease from the first synapse (connecting the first neuron of the convolutional population with the R neuron) to the last (connecting the last neuron of the convolutional population with the R neuron): i.e. the last synapse should have 0 delay, the second *delay_reichardt* and so on. In case of leftward sensitivity, the delay should increase from the first synapse to the last one. Note that, as we fixed the spatial filtering, the delay is what defines the preferred speed of the neurons and must therefore be adapted to the speed of the bar.

In [ ]:
# Adapt the delay to the speed of the input stimulus 
delay_reichardt = {'v1': 6, 'v2': 30, 'v3': 50}.get(speed) * ms

# Synapses: reichardt model - RIGHTWARD motion selectivity
Syn_reichardt_R = # ...

# Synapses: reichardt model - LEFTWARD motion selectivity
Syn_reichardt_L = # ...

Monitor membrane potentials and spikes from the convolutional layer and both output neurons.

In [ ]:
# Write your code here 

Build and feed the network object.

In [ ]:
# Write your code here

Run the simulation for 300ms.

In [ ]:
# Write your code here 

Plot the membrane potentials of all neurons in the convolutional layer and show a rasterplot of all spikes in such layer.

In [ ]:
# Write your code here

**From the rasterplot, you should notice that the moving bar defines an orientation in space-time, where the sign of the slope is related to the direction of the motion and its magnitude is related to the speed.**

Print the time interval between spikes in consecutive neurons of the convolutional layer. If the preferred speed of the output neurons matches the actual speed of the input stimulus, such delays should all be very close to the one that we have set between adjacent spatial filters.

In [ ]:
# Write your code here 

Plot the membrane potentials of the 2 output neurons and show a rasterplot of their spiking activity.

In [ ]:
# Write your code here

**Only the neuron that is sensitive to the same direction of motion as the input bar should fire a spike. In this case, the input spikes from the convolutional layer will be coincident (thanks to the correct temporal filterings) thus summing up and triggering the target neuron. Instead, the inputs of the neuron sensitive to the opposite direction will not be coincident and this detector will thus not reach the firing threshold for emitting a spike.**


---

#### **Extensions**

1. Try to change the direction and speed of the input data and watch the resulting behavior of the network.
2. Try to add some other output neurons sensitive to different speeds of motion to view their response in case of non-preferred input speeds. Particularly, if you set the delay for the first pair of output neurons (R and L) so that their sensitivity is to the speed *v1*, you may add 2 more pairs of neurons sensitive to the speeds *v2* and *v3* respectively. Then change speed and direction of the stimulus and observe the activity of all such neurons. Note: this task is left to the most willing students and the solution will <u>not</u> be given.